In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ratemeter/sample_submission.csv
/kaggle/input/ratemeter/train.csv
/kaggle/input/ratemeter/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score,f1_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import RandomOverSampler
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/w

In [30]:
train_data = pd.read_csv(r'/kaggle/input/ratemeter/train.csv')
train_data

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating
0,1c845473e18c23f917126cb29bc8d243,6547260,5805e560deacf886537b2e76b2653515,I finished this in two days. THIS SERIES IS JU...,Sat Nov 16 00:26:42 -0800 2013,Mon Jan 26 11:14:39 -0800 2015,Mon Jan 26 11:14:39 -0800 2015,Sat Jan 24 00:00:00 -0800 2015,0,0,5
1,0867fa7e18da1a292d0256fad0ee4e31,17201095,84c2e89581004bedd01dea2833bed181,Review to follow \n Goodreads 4 \n Amazon 5 \n...,Wed Jan 02 11:22:44 -0800 2013,Sun Apr 28 20:23:22 -0700 2013,NaN,NaN,0,0,5
2,a57237ef712461148672852d8f47e300,16143347,956d4ea003698d83c1495cc251be5bc0,I will have a full review up eventually but fo...,Thu Jun 19 15:48:04 -0700 2014,Thu May 05 09:09:58 -0700 2016,Thu Jun 19 00:00:00 -0700 2014,Thu Jun 19 00:00:00 -0700 2014,0,0,5
3,80589c77f4fa826eaaf7c5d71ff3b81e,7863423,e5ed7e0401cbe5c44e7b7ee4e1af4f09,4 stars. Aww! I loved this one! I have read th...,Wed Sep 16 18:52:06 -0700 2015,Sun Sep 20 18:34:34 -0700 2015,Sun Sep 20 00:00:00 -0700 2015,Sat Sep 19 00:00:00 -0700 2015,1,0,4
4,916977f433fbf96ae9de9669f8aec040,17262303,51b4449f83ff8c9d13a3b6aa2df4fb27,this is official: Patrick Ness is my fav author,Sun Oct 13 11:47:17 -0700 2013,Mon Sep 15 17:51:03 -0700 2014,Tue Sep 16 17:51:03 -0700 2014,NaN,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...
629995,c658e5a5570ea6f5bab3cb560fc803a9,13581132,9a433cc1b1aa05d6ef8d03d246acb512,"Wow, just ... wow. I absolutely loved this boo...",Wed Apr 25 10:05:51 -0700 2012,Thu Apr 04 09:19:57 -0700 2013,Sun Dec 16 00:00:00 -0800 2012,Thu Dec 13 00:00:00 -0800 2012,0,0,5
629996,8a043ed0e8e8cbf7c4b42331f8803794,20622081,8ff74d58f8700e8a6e1278e4eca6dedb,***Review Coming Soon***,Wed Aug 06 18:33:50 -0700 2014,Wed Aug 06 18:37:15 -0700 2014,Thu Jul 24 00:00:00 -0700 2014,Thu Jul 24 00:00:00 -0700 2014,0,0,3
629997,d13b598d674c7879ce3b978f499cd07a,20443207,5b9de0ff34caaa2076a35820b21edc76,I'm in love with Marie Rutkoski's writing. It'...,Wed Mar 05 22:30:53 -0800 2014,Sun Mar 22 11:14:56 -0700 2015,Fri Mar 20 00:00:00 -0700 2015,NaN,1,0,5
629998,f5c084cbc33f406ae035c298644a9cd4,17447050,882c8fe7b1500746e8fa5fbb64bc1aa6,It was pretty much a walking contradiction. \n...,Fri Aug 16 14:36:18 -0700 2013,Fri Aug 16 14:37:17 -0700 2013,NaN,NaN,0,0,3


In [31]:
test_data = pd.read_csv(r'/kaggle/input/ratemeter/test.csv')
test_data

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,de3a6a28c83cda006b415d45d64674c9,1618,d76ce5becf493e5c653610edb806ffc4,"I'm going to keep this review short, because I...",Tue Jun 09 10:37:48 -0700 2015,Wed Jun 10 13:43:38 -0700 2015,Wed Jun 10 14:08:39 -0700 2015,Tue Jun 09 00:00:00 -0700 2015,8,0
1,d9cfab35a87e32084b1817dfb0e98748,15776309,9c5c9aed79255a1a610dfc153ee90ad6,"You know, I was really stoked to see this come...",Mon Apr 22 09:19:40 -0700 2013,Wed Jul 10 14:34:27 -0700 2013,Wed Jul 10 14:34:27 -0700 2013,Tue Jul 09 00:00:00 -0700 2013,0,0
2,24935a870a46525a37be92775ab18f76,478927,ad26332459cda8f40581fef7a29b800c,This is one of those books where you know you ...,Sat Mar 14 12:16:55 -0700 2015,Sun Apr 05 11:25:00 -0700 2015,Wed Mar 18 00:00:00 -0700 2015,Sat Mar 14 00:00:00 -0700 2015,14,0
3,6ccb40eabdd0db7895aac00963256469,22628,5bb75768be7f2ddcd632336778b67f5a,The perks of being a wallflower \n What does a...,Sat Sep 12 08:43:30 -0700 2015,Sat Sep 12 09:19:29 -0700 2015,NaN,NaN,0,0
4,9f9d0f6e9a6a5797a252ef81abc9421c,13596809,3d7f76ea566b9ce0700772236094d936,"So, I wrote a review for this when I read it b...",Sun Jul 14 19:36:13 -0700 2013,Thu Jun 02 16:45:45 -0700 2016,Sat Mar 23 00:00:00 -0700 2013,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
269995,6cbb612c9de13d3f8c6a7e0aad48fb5d,29844228,0c422ec7786a4d50ed2cc6822a32893d,I wanted to read Thirteen Reasons Why even bef...,Tue May 16 06:58:26 -0700 2017,Tue May 16 07:03:44 -0700 2017,Tue May 16 07:03:44 -0700 2017,NaN,0,0
269996,1b6af0cd383352dcf2267ff62ef313c4,26245098,c5650aa430a4145b00b422339944c641,I had never read a Matthew Quick book before t...,Mon Apr 11 14:10:55 -0700 2016,Tue May 17 15:10:19 -0700 2016,Thu May 12 00:00:00 -0700 2016,Tue May 10 00:00:00 -0700 2016,4,0
269997,54db5f695dbad039288ac0c3d1795ad9,16051395,7accbbf4c7f3e991c6439e11bdd3a94a,I'm not going to enjoy this at all after the f...,Sat Jul 09 01:02:04 -0700 2016,Fri Sep 29 00:23:36 -0700 2017,Fri Sep 29 00:23:36 -0700 2017,Tue Sep 19 06:33:33 -0700 2017,0,0
269998,22622aec2b5c6211184919117897fdc7,20657437,b7dc269df1bbb4464d633dc54ca1c40e,At times I found Tella a little annoying and I...,Fri Aug 29 09:25:45 -0700 2014,Mon Mar 30 09:21:30 -0700 2015,NaN,NaN,0,0


In [32]:
# Text preprocessing 

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join the tokens back into a single string
    processed_text = ' '.join(tokens)
    return processed_text

In [33]:
# Applied text preprocessing to train data
train_data['processed_text'] = train_data['review_text'].apply(preprocess_text)

In [34]:
# Applied text preprocessing to test data
test_data['processed_text'] = test_data['review_text'].apply(preprocess_text)

In [35]:
X_train_text = train_data['processed_text']
y_train_text = train_data['rating']
X_val = test_data['processed_text']

In [36]:
# Splitting the data into training and validation set
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train_text, y_train_text, test_size=0.2, random_state=42)

In [62]:
# Building the model using pipeline here TF-IDF vectrorizer and Logistic Regression model is used
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=5000)), 
    ('clf', LogisticRegression(random_state=42, max_iter=1000, class_weight = 'balanced', C=1))
])

In [63]:
# Training the model
model.fit(X_train_new, y_train_new)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(C=1, class_weight='balanced', max_iter=1000,
                                    random_state=42))])

In [64]:
# Prediction done on the validation set
y_predicted_val = model.predict(X_val)

In [65]:
# Calculate F1 score and accuracy for multiclass classification
f1 = f1_score(y_val, y_predicted_val, average='weighted')
accuracy = accuracy_score(y_val, y_predicted_val)
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

F1 Score: 0.48925168078879055
Accuracy: 0.478484126984127


In [41]:
# from sklearn.model_selection import cross_val_predict
# from sklearn.model_selection import GridSearchCV

In [42]:
# param_grid = {'tfidf__max_features': [1000, 5000, 10000],
#               'clf__C': [0.1, 1, 10]}

In [66]:
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)
# grid_search.fit(X_train_text, y_train_text)
# print("Best Hyperparameters:", grid_search.best_params_)

In [28]:
# # Use the best model for predictions
# best_model = grid_search.best_estimator_
# y_pred_best = best_model.predict(X_val)

# # Calculate F1 score for the best model
# f1_best = f1_score(y_val, y_pred_best, average='weighted')
# print(f"Best Model F1 Score: {f1_best}")



Best Model F1 Score: 0.8077455898747459


In [70]:
# Prediction done on the test set
X_test_text1 = test_data['processed_text']
y_pred = model.predict(X_test_text1)

In [71]:
# Save submission in a csv file using dataframe
submission_df = pd.DataFrame({
    'review_id': test_data['review_id'].values,  # Extract values from pandas Series
    'rating': y_pred
})
submission_df.to_csv('submission7.csv', index=False)